From a slack message by L Grey in the `servicex-cms` chat.

In [15]:
from pathlib import Path
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from dask.distributed import Client
import dask
import dask_awkward
import awkward as ak
import hist.dask as hda

def extract_pushdown(coll):
    hlg_sorted = coll.dask._toposort_layers()
    pushdown_deps = []
    for key in hlg_sorted:
        annotations = coll.dask.layers[key].annotations
        if annotations is not None and "pushdown" in annotations:
            #print(key, coll.dask.layers[key].annotations)
            pushdown_deps = [key] + pushdown_deps
    for dep in pushdown_deps:
        layer = coll.dask.layers[dep]
        fcn = list(layer.dsk.values())[0][0]
        if isinstance(layer, dask_awkward.layers.AwkwardBlockwiseLayer) and not isinstance(layer, dask_awkward.layers.AwkwardInputLayer):
            print(layer.dsk)
            # print(dir(layer))
            print(list(layer.keys()))
            print(dep, fcn.fn)
            # print(dir(fcn))
            # print(fcn.arg_repackers[0])
            print("")
        else:
            print(dep, fcn)

if __name__ == "__main__":
    #client = Client()


    dask.config.set({"awkward.optimization.enabled": True, "awkward.raise-failed-meta": True, "awkward.optimization.on-fail": "raise"})

    with dask.annotate(pushdown="servicex"):
        events = NanoEventsFactory.from_root(
            {
                "nano_dy.root": "Events",
                "nano_dy.root": "Events",
                "nano_dy.root": "Events",
                "nano_dy.root": "Events",
            },
            metadata={"dataset": "nano_dy"},
            schemaclass=NanoAODSchema,
        ).events()

        mask = events.Muon.pt > 30
        events = events[ak.any(mask, axis=1)]
        
    myhist = hda.Hist.new.Regular(50, 0, 2.5, name="abseta").Double()

    myhist.fill(abseta=abs(events.Muon.eta))

    extract_pushdown(myhist)

{'getitem-745471f3280ed33060f37331a1f449d0': (<dask_awkward.lib.core.ArgsKwargsPackedFunction object at 0x0000026DE1A75910>, '__dask_blockwise__0', '__dask_blockwise__1')}
[('getitem-745471f3280ed33060f37331a1f449d0', 0), ('getitem-745471f3280ed33060f37331a1f449d0', 1), ('getitem-745471f3280ed33060f37331a1f449d0', 2), ('getitem-745471f3280ed33060f37331a1f449d0', 3)]
getitem-745471f3280ed33060f37331a1f449d0 <built-in function getitem>

{'any-277ee8e8ef8a2536a86ee3bce40985b9': (<dask_awkward.lib.core.ArgsKwargsPackedFunction object at 0x0000026DE398E9D0>, '__dask_blockwise__0')}
[('any-277ee8e8ef8a2536a86ee3bce40985b9', 0), ('any-277ee8e8ef8a2536a86ee3bce40985b9', 1), ('any-277ee8e8ef8a2536a86ee3bce40985b9', 2), ('any-277ee8e8ef8a2536a86ee3bce40985b9', 3)]
any-277ee8e8ef8a2536a86ee3bce40985b9 <function any at 0x0000026DB2A03F60>

{'greater-9c566e91f11b5fb3417ce3b0b2a0ee86': (<dask_awkward.lib.core.ArgsKwargsPackedFunction object at 0x0000026DE2455890>, '__dask_blockwise__0', '__dask_bloc

c:\Users\gordo\Code\iris-hep\awkward-20-testing\.venv\Lib\site-packages\coffea\nanoevents\schemas\nanoaod.py:243: RuntimeWarning: Missing cross-reference index for FatJet_genJetAK8Idx => GenJetAK8
  warnings.warn(
